In [1]:
import torch
import torch.nn as nn
from CustomLSTM import CustomLSTM
from urllib.request import urlretrieve
import re
import sys
import numpy as np
import random
import html

# Loading data

In [6]:
from torch.utils.data import TensorDataset, DataLoader
import character_level_data as chdata
batch_size = 128

train_data = chdata.get_char_data()
train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size)


Number of total characters: 59711
Number of unique characters: 66
Index - character pairs: {0: '\n', 1: ' ', 2: '!', 3: '&', 4: '(', 5: ')', 6: '+', 7: ',', 8: '-', 9: '.', 10: '/', 11: '0', 12: '1', 13: '2', 14: '3', 15: '4', 16: '5', 17: '6', 18: '7', 19: '8', 20: '9', 21: ':', 22: ';', 23: '=', 24: '?', 25: '[', 26: ']', 27: '_', 28: 'a', 29: 'b', 30: 'c', 31: 'd', 32: 'e', 33: 'f', 34: 'g', 35: 'h', 36: 'i', 37: 'j', 38: 'k', 39: 'l', 40: 'm', 41: 'n', 42: 'o', 43: 'p', 44: 'q', 45: 'r', 46: 's', 47: 't', 48: 'u', 49: 'v', 50: 'w', 51: 'x', 52: 'y', 53: 'z', 54: '{', 55: '|', 56: '}', 57: 'á', 58: 'é', 59: 'í', 60: 'ó', 61: 'õ', 62: 'ö', 63: 'ú', 64: 'û', 65: 'ü'}
Nr. of training data points: 19891
A random example of training point:  hazánk iránt,
azt életében s holtában m e
Shape of training tensor: (19891, 40, 66)
Shape of test tensor: (19891, 66)


# Build model

In [2]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [5]:
model = nn.Sequential(
    CustomLSTM(input_sz = 66, hidden_sz = 128, return_states = False, return_sequences = False),
    nn.Linear(128, 66)).float().to(device)

In [7]:
lr=0.01
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)

# Training

In [ ]:
epochs = 10

model.train()
for i in range(epochs):
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output = model(inputs.float())
        loss = loss_func(output, torch.argmax(labels, dim = 1))
        loss.backward()
        optimizer.step()
   
    print("Epoch: {}/{}...".format(i+1, epochs),
          "Loss: {:.6f}...".format(loss.item()))

# Evaluating

In [13]:
generated = ''
start_index = random.randint(0, len(text) - maxlen - 1)
sentence = text[start_index: start_index + maxlen] # kiválasztunk egy kezdeti szöveget, amiből kiindulunk, a neuronháló ezt fogja folytatni
generated += sentence
softmax = nn.Softmax(dim=1)

In [14]:
sentence = 'rnél nincs semmi csodálatosabb. õ az, ki'
generated = sentence

In [15]:
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

for i in range(400): # Generating 400 consecutive characters
    # creating one-hot encoded input test set
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1
    
    x = torch.from_numpy(x).to(device)
    next_index = torch.argmax(softmax(model(x.float())), dim = 1) # forward pass
    next_char = indices_char[next_index[0].item()] # get predicted character

    generated += next_char
    sentence = sentence[1:] + next_char # add the new character to the end of input sequence of the next step

    sys.stdout.write(next_char) # print generated character

----- Generating with seed: "rnél nincs semmi csodálatosabb. õ az, ki"
rnél nincs semmi csodálatosabb. õ az, ki minden telidésztéli szenvészt - a apzt az alt a talott az alt meg meg a taror utón; ki azt akarllótt a kell csak azak a szelni ezt a szennél az és az én szoldottal el.
s akarva apzt a szennél égért a karatt a szélteltest a karait szó, alapurengedek szót azt akkor én iszenységeik - a arcra tellát a apy nem karai szó, alapul szenvészt a azt a karatt a szélnyezékhal jóslállaniasz alyan olatt szenvés